In [2]:
import pandas as pd
import numpy as np

In [3]:
filepath = './data/raw_data.csv'
raw_data = pd.read_csv(filepath)
raw_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country,Capital Gains
0,2025-04-10 00:00:00-04:00,5.610000,5.660000,5.280000,5.500000,9642800.0,0.0,0.0,peloton,PTON,fitness,usa,NaN
1,2025-04-10 00:00:00-04:00,91.000000,92.949997,85.059998,88.699997,65121200.0,0.0,0.0,amd,AMD,technology,usa,NaN
2,2025-04-10 00:00:00-04:00,111.120003,112.220001,108.099998,110.510002,106100.0,0.0,0.0,adidas,ADDYY,apparel,germany,NaN
3,2025-04-10 00:00:00-04:00,255.020004,255.500000,238.520004,246.889999,4795200.0,0.0,0.0,american express,AXP,finance,usa,NaN
4,2025-04-10 00:00:00-04:00,21.346001,21.600000,21.346001,21.575001,500.0,0.0,0.0,puma,PMMAF,apparel,germany,NaN


In [4]:
raw_data.drop(columns=['Dividends', 'Stock Splits', 'Country', 'Capital Gains'], inplace=True)
raw_data.head()

,Date,Open,High,Low,Close,Volume,Brand_Name,Ticker,Industry_Tag
0,2025-04-10 00:00:00-04:00,5.610000,5.660000,5.280000,5.500000,9642800.0,peloton,PTON,fitness
1,2025-04-10 00:00:00-04:00,91.000000,92.949997,85.059998,88.699997,65121200.0,amd,AMD,technology
2,2025-04-10 00:00:00-04:00,111.120003,112.220001,108.099998,110.510002,106100.0,adidas,ADDYY,apparel
3,2025-04-10 00:00:00-04:00,255.020004,255.500000,238.520004,246.889999,4795200.0,american express,AXP,finance
4,2025-04-10 00:00:00-04:00,21.346001,21.600000,21.346001,21.575001,500.0,puma,PMMAF,apparel


In [5]:
companies = raw_data[['Brand_Name', 'Ticker', 'Industry_Tag']].drop_duplicates()
print(companies.head())
companies.shape

         Brand_Name Ticker Industry_Tag
0           peloton   PTON      fitness
1               amd    AMD   technology
2            adidas  ADDYY      apparel
3  american express    AXP      finance
4              puma  PMMAF      apparel


(61, 3)

## Prepare processed data

In [6]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

processed_data = raw_data.copy()

processed_data['Date'] = pd.to_datetime(processed_data['Date'], utc=True)

# Encode industry tags and tickers using LabelEncoder
label_encoder = LabelEncoder()
processed_data['Industry_Tag_Encoded'] = label_encoder.fit_transform(processed_data['Industry_Tag'])
processed_data['Ticker_Encoded'] = label_encoder.fit_transform(processed_data['Ticker'])

# # Normalize the encoded values to a range between 0 and 1
# scaler = MinMaxScaler()
# processed_data['Industry_Tag_Normalized'] = scaler.fit_transform(processed_data[['Industry_Tag_Encoded']])
# processed_data['Ticker_Normalized'] = scaler.fit_transform(processed_data[['Ticker_Encoded']])

# normalize values between 0 and 1
scaler = MinMaxScaler()
processed_data['open_normalized'] = scaler.fit_transform(processed_data[['Open']])
processed_data['high_normalized'] = scaler.fit_transform(processed_data[['High']])
processed_data['low_normalized'] = scaler.fit_transform(processed_data[['Low']])
processed_data['close_normalized'] = scaler.fit_transform(processed_data[['Close']])
processed_data['volume_normalized'] = scaler.fit_transform(processed_data[['Volume']])

processed_data.head()


,Date,Open,High,Low,Close,Volume,Brand_Name,Ticker,Industry_Tag,Industry_Tag_Encoded,Ticker_Encoded,open_normalized,high_normalized,low_normalized,close_normalized,volume_normalized
0,2025-04-10 04:00:00+00:00,5.610000,5.660000,5.280000,5.500000,9642800.0,peloton,PTON,fitness,9,46,0.001571,0.001577,0.001510,0.001547,1.299281e-03
1,2025-04-10 04:00:00+00:00,91.000000,92.949997,85.059998,88.699997,65121200.0,amd,AMD,technology,22,5,0.026355,0.026784,0.025185,0.025822,8.774502e-03
2,2025-04-10 04:00:00+00:00,111.120003,112.220001,108.099998,110.510002,106100.0,adidas,ADDYY,apparel,0,3,0.032195,0.032349,0.032022,0.032185,1.429603e-05
3,2025-04-10 04:00:00+00:00,255.020004,255.500000,238.520004,246.889999,4795200.0,american express,AXP,finance,7,7,0.073961,0.073725,0.070724,0.071976,6.461105e-04
4,2025-04-10 04:00:00+00:00,21.346001,21.600000,21.346001,21.575001,500.0,puma,PMMAF,apparel,0,44,0.006138,0.006180,0.006277,0.006237,6.737055e-08


In [7]:
# group by brand name and sort by date
sorted_data = processed_data.groupby('Brand_Name').apply(lambda x: x.sort_values('Date'), include_groups=False).reset_index(drop=True)
sorted_data = sorted_data.drop(columns=['Industry_Tag', 'Ticker'])
sorted_data.head()

,Date,Open,High,Low,Close,Volume,Industry_Tag_Encoded,Ticker_Encoded,open_normalized,high_normalized,low_normalized,close_normalized,volume_normalized
0,2000-01-03 05:00:00+00:00,25.215814,25.330655,24.690826,24.772856,2173400.0,18,35,0.007262,0.007257,0.007270,0.007170,0.000293
1,2000-01-04 05:00:00+00:00,24.379101,24.887683,23.788490,23.788490,2713800.0,18,35,0.007019,0.007129,0.007002,0.006883,0.000366
2,2000-01-05 05:00:00+00:00,23.919738,25.265018,23.919738,24.477537,3699400.0,18,35,0.006885,0.007238,0.007041,0.007084,0.000498
3,2000-01-06 05:00:00+00:00,24.756433,26.905600,24.756433,26.446236,5975800.0,18,35,0.007128,0.007712,0.007289,0.007658,0.000805
4,2000-01-07 05:00:00+00:00,26.544662,27.250114,26.232951,26.971214,4101200.0,18,35,0.007647,0.007811,0.007727,0.007811,0.000553


In [9]:
# create a lookup table for the industry tags and tickers
company_lookup = processed_data[['Brand_Name', 'Ticker', 'Industry_Tag', 'Ticker_Encoded', 'Industry_Tag_Encoded']].drop_duplicates()
company_lookup.reset_index(drop=True, inplace=True)
company_lookup.head()

,Brand_Name,Ticker,Industry_Tag,Ticker_Encoded,Industry_Tag_Encoded
0,peloton,PTON,fitness,46,9
1,amd,AMD,technology,5,22
2,adidas,ADDYY,apparel,3,0
3,american express,AXP,finance,7,7
4,puma,PMMAF,apparel,44,0


In [10]:
company_lookup.to_csv('./data/processed/company_lookup.csv', index=False)
sorted_data.to_csv('./data/processed/processed_data.csv', index=False)